<a href="https://colab.research.google.com/github/keonju2/2022_DL_project/blob/main/yolov5_%ED%9A%A8%EC%A4%80_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Trial #8 ==> 성공
* --img 640
* --batch 32
* --weights **yolov5m**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install
!pip install -q roboflow

import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


Cloning into 'yolov5'...
remote: Enumerating objects: 12093, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 12093 (delta 23), reused 16 (delta 7), pack-reused 12047
Receiving objects: 100% (12093/12093), 11.92 MiB | 18.58 MiB/s, done.
Resolving deltas: 100% (8381/8381), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 13.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 145 kB 14.4 MB/s 
     |████████████████████████████████| 178 kB 65.9 MB/s 
     |████████████████████████████████| 1.1 MB 62.4 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 138 kB 

In [3]:
from zipfile import ZipFile
import os

def unzip(source_file, dest_path):
    with ZipFile(source_file, 'r') as zf:
        zipInfo = zf.infolist()
        for member in zipInfo:
            try:
                member.filename = member.filename.encode('cp437').decode('euc-kr', 'ignore')
                zf.extract(member, dest_path)
            except:
                raise Exception('what?!')

In [4]:
file_names = '/content/gdrive/MyDrive/DLP/Dataset/Training/[원천]도심로.zip'
unzip(file_names,'/content/yolov5/custom_dataset/train/images')

In [5]:
file_names = '/content/gdrive/MyDrive/DLP/Dataset/Valid/원천데이터.zip'
unzip(file_names,'/content/yolov5/sample_data')

In [6]:
shutil.move('/content/yolov5/sample_data/bb',
            '/content/yolov5/custom_dataset/val/images')

'/content/yolov5/custom_dataset/val/images'

In [7]:
shutil.rmtree('/content/yolov5/sample_data')

In [8]:
file_names = '/content/gdrive/MyDrive/DLP/Dataset/Valid/bb.zip'
unzip(file_names,'/content/yolov5/sample_data')

In [9]:
shutil.move('/content/yolov5/sample_data/bb','/content/yolov5/custom_dataset/val/labels')

'/content/yolov5/custom_dataset/val/labels'

In [10]:
file_names = '/content/gdrive/MyDrive/DLP/Dataset/Training/labels.zip'
unzip(file_names,'/content/yolov5/sample_data')

In [11]:
shutil.move('/content/yolov5/sample_data/labels',
            '/content/yolov5/custom_dataset/train/labels')

'/content/yolov5/custom_dataset/train/labels'

In [12]:
import yaml

make_yaml = {
             "train" : "/content/yolov5/custom_dataset/train",
            "val": "/content/yolov5/custom_dataset/val",

             "nc" : 32,
             "names": ['Vehicle_Car', 'Vehicle_Bus', 'Vehicle_Motorcycle', 'Vehicle_Unknown', 
                       'Pedestrian_Pedestrian', 'Pedestrian_Bicycle', 'Lane_White_Dash', 'Lane_White_Solid', 
                       'Lane_Yellow_Dash', 'Lane_Yellow_Solid', 'Lane_Blue_Dash', 'Lane_Blue_Solid', 
                       'TrafficLight_Red', 'TrafficLight_Yellow', 'TrafficLight_Green', 'TrafficLight_Arrow', 
                       'TrafficLight_RedArrow', 'TrafficLight_YellowArrow', 'TrafficLight_GreenArrow', 
                       'TrafficSign_Speed', 'TrafficSign_Else', 'RoadMark_StopLine', 'RoadMark_Crosswalk', 
                       'RoadMark_Number', 'RoadMark_Character', 'RoadMarkArrow_Straight', 'RoadMarkArrow_Left', 
                       'RoadMarkArrow_Right', 'RoadMarkArrow_StraightLeft', 'RoadMarkArrow_StraightRight', 
                       'RoadMarkArrow_Uturn', 'RoadMarkArrow_Else']
}
with open('/content/yolov5/data/custom_dataset.yaml', 'w') as outfile:
    yaml.dump(make_yaml, outfile, default_flow_style=False)

In [13]:
lab =['Vehicle_Car', 'Vehicle_Bus', 'Vehicle_Motorcycle', 'Vehicle_Unknown', 
                       'Pedestrian_Pedestrian', 'Pedestrian_Bicycle', 'Lane_White_Dash', 'Lane_White_Solid', 
                       'Lane_Yellow_Dash', 'Lane_Yellow_Solid', 'Lane_Blue_Dash', 'Lane_Blue_Solid', 
                       'TrafficLight_Red', 'TrafficLight_Yellow', 'TrafficLight_Green', 'TrafficLight_Arrow', 
                       'TrafficLight_RedArrow', 'TrafficLight_YellowArrow', 'TrafficLight_GreenArrow', 
                       'TrafficSign_Speed', 'TrafficSign_Else', 'RoadMark_StopLine', 'RoadMark_Crosswalk', 
                       'RoadMark_Number', 'RoadMark_Character', 'RoadMarkArrow_Straight', 'RoadMarkArrow_Left', 
                       'RoadMarkArrow_Right', 'RoadMarkArrow_StraightLeft', 'RoadMarkArrow_StraightRight', 
                       'RoadMarkArrow_Uturn', 'RoadMarkArrow_Else']

In [14]:
with open('/content/yolov5/w+labels.txt','w') as f:
  for i in lab:
    if i != lab[-1]:
      f.write(i+'\n')
    else:
      f.write(i)

In [15]:
for (root, directories, files) in os.walk('/content/yolov5/custom_dataset'):
    for file in files:
        file_path = os.path.join(root, file)
        change_file_path = file.replace('_v001_1','')
        file_path2 = os.path.join(root, change_file_path)
        os.rename(file_path,file_path2)

In [16]:
for (root, directories, files) in os.walk('/content/yolov5/custom_dataset'):
    for file in directories:
        file_path = os.path.join(root, file)
        if 'annotations' in file_path:
          change_file_path = file_path.replace('_annotations_v001_1','')
          
          shutil.move(file_path,change_file_path)
        

In [17]:
torch.cuda.empty_cache()

In [18]:
# batch 64,32,16
# img 1280 / 640

!python train.py --img 640 --batch 32 --epochs 5 --data /content/yolov5/data/custom_dataset.yaml --weights yolov5m.pt      

train: weights=yolov5m.pt, cfg=, data=/content/yolov5/data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-234-gd490bdd Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv

In [19]:
save_path = '/content/gdrive/MyDrive/DLP/result/효준/'
save_file_path = '/content/yolov5/runs/train/'

shutil.move(save_file_path, save_path)

'/content/gdrive/MyDrive/DLP/result/효준/train'

In [20]:
!python detect.py --source '/content/gdrive/MyDrive/DLP/시연영상.mp4' --weight '/content/gdrive/MyDrive/DLP/result/효준/train/exp/weights/best.pt'

detect: weights=['/content/gdrive/MyDrive/DLP/result/효준/train/exp/weights/best.pt'], source=/content/gdrive/MyDrive/DLP/시연영상.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-234-gd490bdd Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 290 layers, 20978205 parameters, 0 gradients, 48.3 GFLOPs
video 1/1 (1/557) /content/gdrive/.shortcut-targets-by-id/1tuDEf-stZClDnTJwrYx_rD8asx92xcnr/DLP/시연영상.mp4: 384x640 11 Vehicle_Cars, 3 Vehicle_Buss, 1 Vehicle_Unknown, 8 Pedestrian_Pedestrians, 3 TrafficLight_Reds, 3 RoadMark_Crosswalks, Done. (0.018s)
video 1/1 (2/557) /content/gd